In [1]:
library(tidyverse)
library(magrittr)
library(here)
library(phangorn)
library(furrr)

devtools::load_all(".")

plan(multiprocess)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

here() starts at /mnt/expressions/mp/archaic-ychr
Loading required package: ape
Loading required package: future
Loading ychr


### Total number of diagnostic sites (derived in non-Africans)?

In [2]:
highcov <- read_vcf(here("data/vcf/full_modern.vcf.gz"), mindp = 3, maxdp = 0.98, var_only = TRUE)

In [3]:
samples <- read_info(highcov)
outgroup <- c("a00", "S_Ju_hoan_North_1")
ingroup <- filter(samples, pop %in% c("WestEur", "EastEur"))$name

In [4]:
# define diagnostic positions (outgroup pops different from ingroup pops)
outgroup_pos <- rowMeans(highcov[, outgroup])
ingroup_pos <- rowMeans(highcov[, ingroup], na.rm = T)

info_highcov <- highcov[abs(outgroup_pos - ingroup_pos) == 1, ] %>% filter(!is.na(chrom))

In [5]:
nrow(info_highcov)

[1] 268

### Count non-African derived SNPs in each individual

In [6]:
estimate_cont <- function(individual, gt) {
    # get names of outgroup and ingroup individuals
    samples <- read_info(gt)
    outgroup <- c("a00", "S_Ju_hoan_North_1")
    ingroup <- filter(samples, pop %in% c("WestEur", "EastEur"))$name

    # subset only to relevant individuals
    gt <- gt[, c("chrom", "pos", "REF", "ALT", unique(c(individual, outgroup, ingroup)))]

    # define diagnostic positions (outgroup pops different from ingroup pops)
    outgroup_pos <- rowMeans(gt[, outgroup])
    ingroup_pos <- rowMeans(gt[, ingroup], na.rm = T)
    info_gt <- gt[abs(outgroup_pos - ingroup_pos) == 1, ]

    # extract a vector of alleles for the tested individual
    alleles <- info_gt[[individual]] %>% .[!is.na(.)]

    # convert the vector of alleles to counts of ancestral and derived alleles
    counts <- factor(alleles, levels = c(0, 1)) %>% table %>% as.data.frame
    names(counts) <- c("allele", "count")
    counts %>%
        mutate(allele = ifelse(allele == 0, "derived", "ancestral")) %>%
        spread(allele, count) %>%
        mutate(name = fix_name(individual),
               total = length(alleles),
               derived / total) %>%
        select(name, everything())
}

In [7]:
counts <- future_map_dfr(c("spy1", "mez2", "den4", "den8", "elsidron2"), function(individual) {
    capture <- ifelse(individual == "elsidron2", "lippold", "full")

    gt <- read_genotypes(individual, capture, mindp = 3, maxdp = 0.98, var_only = TRUE)

    estimate_cont(individual, gt)
})

In [8]:
counts

name,ancestral,derived,total,derived/total
<chr>,<int>,<int>,<int>,<dbl>
Spy 94a,15,1,16,0.0625
Mezmaiskaya 2,189,0,189,0.0000
Denisova 4,14,0,14,0.0000
Denisova 8,90,0,90,0.0000
El Sidrón 1253 (560 kb),29,0,29,0.0000


### Sanity check - derived-allele counts in present-day SGDP samples

In [9]:
counts_sgdp <- samples$name %>% future_map_dfr(~ estimate_cont(.x, highcov))

In [10]:
counts_sgdp %>% arrange(derived)

name,ancestral,derived,total,derived/total
<chr>,<int>,<int>,<int>,<dbl>
S_Ju_hoan_North_1,268,0,268,0.0000000
A00,268,0,268,0.0000000
A00-1,249,0,249,0.0000000
A00-2,265,0,265,0.0000000
S_Dinka_1,106,153,259,0.5907336
S_Mandenka_1,107,156,263,0.5931559
S_Gambian_1,106,158,264,0.5984848
S_Yoruba_2,107,159,266,0.5977444
S_Mbuti_1,105,160,265,0.6037736


# Estimate contamination based on pileups of reads

In [11]:
info_pos <- info_highcov %>% select(chrom, pos, REF, ALT)

### Ancient individuals

In [12]:
arch_pileups <- future_map_dfr(c("spy1", "mez2", "den4", "den8", "elsidron2"), function(individual) {
    capture <- ifelse(individual == "elsidron2", "lippold", "full")
    df_pileup <- read_tsv(here(paste0("data/pileup/", capture, "_", individual, ".txt.gz")),
                          col_type = "cicciiii")

    left_join(info_pos, df_pileup, by = c("chrom", "pos")) %>%
        select(-ref) %>%
        filter(nchar(pileup) >= 3) %>%
        mutate(
            prop = case_when(
                REF == "A" ~ A / (A + C + G + T),
                REF == "C" ~ C / (A + C + G + T),
                REF == "G" ~ G / (A + C + G + T),
                REF == "T" ~ T / (A + C + G + T)
            ),
            name = fix_name(individual)
        )
    }
)

In [13]:
arch_pileups %>%
    group_by(name) %>%
    summarise(cont = weighted.mean(prop, nchar(pileup)), total = n()) %>%
    arrange(-cont)

name,cont,total
<chr>,<dbl>,<int>
Denisova 4,0.19469027,32
Spy 94a,0.10144928,20
Mezmaiskaya 2,0.03891051,243
Denisova 8,0.03629032,110
El Sidrón 1253 (560 kb),0.03164557,35


### Present-day humans (sanity check)

In [14]:
modern_pileups <- future_map_dfr(c("a00", "S_French-1", "S_Saami-2", "S_Han-2"), function(individual) {
    capture <- ifelse(individual == "elsidron2", "lippold", "full")
    df_pileup <- read_tsv(here(paste0("data/pileup/", capture, "_", individual, ".txt.gz")),
                          col_type = "cicciiii")

    left_join(info_pos, df_pileup, by = c("chrom", "pos")) %>%
        select(-ref) %>%
        filter(nchar(pileup) >= 3) %>%
        mutate(
            prop = case_when(
                REF == "A" ~ A / (A + C + G + T),
                REF == "C" ~ C / (A + C + G + T),
                REF == "G" ~ G / (A + C + G + T),
                REF == "T" ~ T / (A + C + G + T)
            ),
            name = fix_name(individual)
        )
    }
)

In [15]:
modern_pileups %>%
    group_by(name) %>%
    summarise(cont = weighted.mean(prop, nchar(pileup)), total = n()) %>%
    arrange(-cont)

name,cont,total
<chr>,<dbl>,<int>
S_Saami-2,1.0000000000,268
S_French-1,0.9998144368,268
S_Han-2,0.9998000400,268
A00,0.0008818342,268


In [16]:
chag_pileups <- future_map_dfr(paste0("chag", c("01", "02", "06", "07", "09", "1141", "13", "14", "19", "41")),
                                    function(individual) {
    capture <- ifelse(individual == "elsidron2", "lippold", "full")
    df_pileup <- read_tsv(paste0("../../chag-ychr/data/pileup/", capture, "_", individual, ".txt.gz"),
                          col_type = "cicciiii")

    left_join(info_pos, df_pileup, by = c("chrom", "pos")) %>%
        select(-ref) %>%
        filter(nchar(pileup) >= 3) %>%
        mutate(
            prop = case_when(
                REF == "A" ~ A / (A + C + G + T),
                REF == "C" ~ C / (A + C + G + T),
                REF == "G" ~ G / (A + C + G + T),
                REF == "T" ~ T / (A + C + G + T)
            ),
            name = fix_name(individual)
        )
    }
)

In [17]:
chag_pileups %>%
    group_by(name) %>%
    summarise(cont = weighted.mean(prop, nchar(pileup)), total = n()) %>%
    arrange(-cont)

name,cont,total
<chr>,<dbl>,<int>
chag01,0.24503311,41
chag09,0.14285714,2
chag19,0.06976744,13
chag41,0.06002144,149
chag07,0.01818182,17
chag13,0.01369863,20
chag02,0.00000000,4
chag1141,0.00000000,1
